In [5]:
import cv2
import numpy as np

def count_spots(image_path):
    # Step 1: Load the image
    image = cv2.imread(image_path)
    
    # Step 2: Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Step 3: Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)
    
    # Step 4: Apply a binary threshold to the image
    _, thresh = cv2.threshold(blurred, 120, 255, cv2.THRESH_BINARY_INV)
    
    # Step 5: Find contours (which represent the spots)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Count the number of spots (contours)
    number_of_spots = len(contours)
    
    # Draw contours on the original image
    output_image = image.copy()
    cv2.drawContours(output_image, contours, -1, (0, 255, 0), 2)
    
    # Display the original image with spots highlighted
    cv2.imshow("Detected Spots", output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return number_of_spots

# Example usage
image_path = '360_F_457474202_N4CKJyXw5AIIzOF7YNW9d71GoGmXDfqo.jpg'  # Replace with the path to your leaf image
spots_count = count_spots(image_path)
print(f"Number of spots detected: {spots_count}")


Number of spots detected: 7


SearchBox

In [12]:
import cv2
import numpy as np

def is_scar_color(pixel, scar_color_lower, scar_color_upper):
    """Check if the given pixel color is within the scar color range."""
    return np.all(pixel >= scar_color_lower) and np.all(pixel <= scar_color_upper)

def count_scar_spots(image_path, window_size=2, step_size=2, scar_color_lower=(76, 68, 58), scar_color_upper=(244, 243, 245)):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    
    # Create a copy of the image for visualization
    output_image = image.copy()

    # Initialize a counter for the number of scars
    scar_count = 0

    # Loop over the image using a sliding window
    for y in range(0, height - window_size, step_size):
        for x in range(0, width - window_size, step_size):
            # Extract the region of interest (window)
            window = image[y:y + window_size, x:x + window_size]
            
            # Check if the window contains scar-like colors
            found_scar = False
            for row in window:
                for pixel in row:
                    if is_scar_color(pixel, scar_color_lower, scar_color_upper):
                        found_scar = True
                        break  # Stop scanning this window if a match is found
            
            if found_scar:
                # Increment the scar counter
                scar_count += 1

                # Draw a rectangle around the detected scar region
                cv2.rectangle(output_image, (x, y), (x + window_size, y + window_size), (0, 255, 0), 2)

    # Save the output image with marked scars
    output_path = 'scars_detected_output.png'
    cv2.imwrite(output_path, output_image)
    
    # Optionally, display the output image
    cv2.imshow('Scars Detected', output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return scar_count, output_path

# Example usage
image_path = 'OneLeaf.PNG'  # Replace with the path to your image
scar_color_lower = (76, 68, 58)  # Lower BGR bound for scar detection (darker values)
scar_color_upper = (244, 243, 245)  # Upper BGR bound for scar detection (lighter values)

scar_count, output_image_path = count_scar_spots(image_path, window_size=20, step_size=10, scar_color_lower=scar_color_lower, scar_color_upper=scar_color_upper)

print(f"Number of scars detected: {scar_count}")
print(f"Output image saved at: {output_image_path}")


Number of scars detected: 1515
Output image saved at: scars_detected_output.png


In [13]:
import cv2
import numpy as np

def is_scar_color(pixel, scar_color_lower, scar_color_upper):
    """Check if the given pixel color is within the scar color range."""
    return np.all(pixel >= scar_color_lower) and np.all(pixel <= scar_color_upper)

def count_scar_spots(image_path, window_size=20, step_size=10, scar_color_lower=(76, 68, 58), scar_color_upper=(244, 243, 245)):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    
    # Convert the image to grayscale and apply thresholding to segment the leaf
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours (the boundary of the leaf)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest contour which will likely be the leaf's boundary
    leaf_contour = max(contours, key=cv2.contourArea)

    # Create a mask to isolate the leaf region
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, [leaf_contour], -1, 255, thickness=cv2.FILLED)
    
    # Create an output image for visualization
    output_image = image.copy()

    # Initialize a counter for the number of scars
    scar_count = 0

    # Loop over the image using a sliding window, but only process pixels inside the leaf contour
    for y in range(0, height - window_size, step_size):
        for x in range(0, width - window_size, step_size):
            # Only process the window if it's inside the leaf boundary
            if np.any(mask[y:y + window_size, x:x + window_size] == 255):  # Check if the window is inside the leaf
                window = image[y:y + window_size, x:x + window_size]
                
                found_scar = False
                for row in window:
                    for pixel in row:
                        if is_scar_color(pixel, scar_color_lower, scar_color_upper):
                            found_scar = True
                            break  # Stop scanning this window if a match is found
                
                if found_scar:
                    # Increment the scar counter
                    scar_count += 1

                    # Draw a rectangle around the detected scar region
                    cv2.rectangle(output_image, (x, y), (x + window_size, y + window_size), (0, 255, 0), 2)

    # Draw the leaf contour on the output image
    cv2.drawContours(output_image, [leaf_contour], -1, (0, 0, 255), 3)

    # Save the output image with marked scars and leaf outline
    output_path = 'scars_and_leaf_output.png'
    cv2.imwrite(output_path, output_image)

    # Optionally, display the output image
    cv2.imshow('Leaf with Scars Detected', output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return scar_count, output_path

# Example usage
image_path = 'OneLeaf.PNG'  # Replace with the path to your image
scar_color_lower = (76, 68, 58)  # Lower BGR bound for scar detection
scar_color_upper = (244, 243, 245)  # Upper BGR bound for scar detection

scar_count, output_image_path = count_scar_spots(image_path, window_size=20, step_size=10, scar_color_lower=scar_color_lower, scar_color_upper=scar_color_upper)

print(f"Number of scars detected: {scar_count}")
print(f"Output image saved at: {output_image_path}")


Number of scars detected: 1477
Output image saved at: scars_and_leaf_output.png


In [14]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('OneLeaf.PNG')

# Convert to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Enhance contrast using CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
contrast_image = clahe.apply(gray_image)

# Thresholding to separate lighter colors from darker colors
# You can adjust the threshold value (e.g., 127) as needed
_, thresholded_image = cv2.threshold(contrast_image, 127, 255, cv2.THRESH_BINARY)

# Show the results
cv2.imshow('Original Image', image)
cv2.imshow('Grayscale Image', gray_image)
cv2.imshow('Contrast Enhanced Image', contrast_image)
cv2.imshow('Thresholded Image', thresholded_image)

# Wait until a key is pressed and close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('Leaf_1.png')

# Convert to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian Blur to smooth the image and reduce details like veins
blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

# Alternatively, you can use Median Blur
# blurred_image = cv2.medianBlur(gray_image, 5)

# Enhance contrast using CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
contrast_image = clahe.apply(blurred_image)

# Thresholding to separate lighter colors from darker colors
# Adjust the threshold value (e.g., 127) as needed
_, thresholded_image = cv2.threshold(contrast_image, 127, 255, cv2.THRESH_BINARY)

# Show the results
cv2.imshow('Original Image', image)
cv2.imshow('Grayscale Image', gray_image)
cv2.imshow('Blurred Image', blurred_image)
cv2.imshow('Contrast Enhanced Image', contrast_image)
cv2.imshow('Thresholded Image', thresholded_image)

# Wait until a key is pressed and close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np
import os

# Load the image
image = cv2.imread('Leaf_1.PNG')

# Convert to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian Blur to smooth the image
blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

# Thresholding to create a mask for the scars
_, mask = cv2.threshold(blurred_image, 200, 255, cv2.THRESH_BINARY)

# Use morphological operations to refine the mask if needed
kernel = np.ones((5, 5), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)  # Close small holes in the mask

# Inpaint the image using the mask
inpainted_image = cv2.inpaint(image, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

# Specify the directory to save the images
save_directory = r'Leaf-Scarring-Project-main\Individual_Code\Henko Holl'

# Ensure the directory exists
os.makedirs(save_directory, exist_ok=True)

# Save the images
cv2.imwrite(os.path.join(save_directory, 'Original_Image.png'), image)
cv2.imwrite(os.path.join(save_directory, 'Mask_for_Scars.png'), mask)
cv2.imwrite(os.path.join(save_directory, 'Inpainted_Image.png'), inpainted_image)

# Show the results
cv2.imshow('Original Image', image)
cv2.imshow('Mask for Scars', mask)
cv2.imshow('Inpainted Image', inpainted_image)

# Wait until a key is pressed and close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np
import os

# Load the mask image (output from the previous program)
mask_image_path = r'Leaf_1.png'
mask = cv2.imread(mask_image_path, cv2.IMREAD_GRAYSCALE)

# Find contours of the scars in the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create an empty image to draw contours
contour_image = np.zeros_like(mask)

# Draw contours and count them
scar_count = 0
for contour in contours:
    if cv2.contourArea(contour) > 100:  # Filter out small contours if needed
        scar_count += 1
        cv2.drawContours(contour_image, [contour], -1, (255), 2)  # Draw contours in white

# Specify the directory to save the images
save_directory = r'Leaf-Scarring-Project-main\Individual_Code\Henko Holl'

# Ensure the directory exists
os.makedirs(save_directory, exist_ok=True)

# Save the mask and the image with contours
cv2.imwrite(os.path.join(save_directory, 'Mask_for_Scars.png'), mask)
cv2.imwrite(os.path.join(save_directory, 'Contours_on_Mask.png'), contour_image)

# Display the scar count on the contour image
cv2.putText(contour_image, f'Scar Count: {scar_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255), 2)

# Show the results
cv2.imshow('Mask for Scars', mask)
cv2.imshow('Contours on Mask', contour_image)

# Wait until a key is pressed and close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('Leaf_1.png')

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur with a smaller kernel
blurred = cv2.GaussianBlur(gray, (3, 3), 0)

# Apply adaptive thresholding
mask = cv2.adaptiveThreshold(blurred, 255, 
                             cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                             cv2.THRESH_BINARY_INV, 
                             11, 2)

# Optionally, apply morphological operations to clean up the mask
kernel = np.ones((3, 3), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

# Apply the mask to the original image
masked_image = cv2.bitwise_and(image, image, mask=mask)

# Display the original and masked images
cv2.imshow('Original Image', image)
cv2.imshow('Masked Image', masked_image)

# Save the masked image
cv2.imwrite('masked_leaf_image.jpg', masked_image)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np

img = cv2.imread("OneLeaf.png", cv2.IMREAD_GRAYSCALE)
scale_percent = 60 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
  
# resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

# threshold to binary
thresh = cv2.threshold(resized, 210, 255, cv2.THRESH_BINARY_INV)[1]  # the 2nd parameter should be changed.

# apply morphology
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4,4))
morph = cv2.morphologyEx(thresh, cv2.MORPH_ERODE, kernel, 1)

letter = morph.copy()
cntrs = cv2.findContours(morph, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  
# cntrs = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  

cntrs = cntrs[0] if len(cntrs) == 2 else cntrs[1]
# cntrs = cntrs[0]

for c in cntrs:
    area = cv2.contourArea(c)
    print(area)
    if area < 100:
        cv2.drawContours(letter,[c],0,(0,0,0),-1)


# do canny edge detection
edges = cv2.Canny(letter, 200, 200) # the result for edges is good.
length = cv2.arcLength(cntrs[0], False)  # not closed curves
print('length = ',length)  # both length and area need calibration

area = cv2.contourArea(cntrs[0])
print('area = ',area)

# Outputs
print(np.squeeze(cntrs[0]), '\n')                    # Contour
print('Contour points:', cntrs[0].shape[0], '\n')
print('arcLength:', cv2.arcLength(cntrs[0], True))  # closed curves

# write results
# cv2.imwrite("K_thresh.png", thresh)    

# show results
# cv2.imshow("K_thresh", thresh)
# cv2.imshow("K_morph", morph)
cv2.imshow("K_letter", letter)
cv2.imshow("K_edges", edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

0.0
0.0
48628.5
77.5
29.0
43.0
99.0
43.0
19.0
30.0
19.0
182.5
24.0
64.0
50.0
221.0
698.5
0.0
0.0
0.0
64.5
44.0
73.0
197.5
126.0
197.5
0.0
34.5
152.0
195.0
37.0
637.0
19.0
26.0
32.0
103.5
117.5
523.5
19.0
51.5
37.0
length =  0.0
area =  0.0
[91 11] 

Contour points: 1 

arcLength: 0.0
